In [71]:
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
import csv

def write_csv(log_filename, csv_filename):
    with open(log_filename, 'r') as log_file:
         log_lines = log_file.readlines()
    with open(csv_filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        for item in log_lines:
            csv_writer.writerow([item])

def sensor_processing(df):
    split_data = df['2023-07-16 17:58:48;536;261;4408;154,0\n'].str.split(';', expand=True)
    split_data.columns = ['date', 'x', 'y', 'z', 'extra1', 'extra2', 'extra3']
    split_data.drop(columns=['extra1', 'extra2', 'extra3'], inplace=True)
    
    split_data['date'] = pd.to_datetime(split_data['date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    split_data['x'] = split_data['x'].str.replace(',', '.').str.strip('\n')
    split_data.dropna(inplace=True)
    split_data['y'] = split_data['y'].str.replace(',', '.').str.strip('\n')
    split_data.dropna(inplace=True)
    split_data['z'] = split_data['z'].str.replace(',', '.').str.strip('\n')
    split_data.dropna(inplace=True)
    
    split_data['x'] = split_data['x'].astype(np.float32)
    split_data['y'] = split_data['y'].astype(np.float32)
    split_data['z'] = split_data['z'].astype(np.float32)
    return split_data

def conversion(df):
    """
    Перевод из мТл в нТл
    """
    df['x'] = [x*0.013 for x in df['x']] 
    df['y'] = [y*0.013 for y in df['y']] 
    df['z'] = [z*0.013 for z in df['z']] 
    return df

def split_datetime(extracted_time):
    date = extracted_time.split(' ')[0]
    time = extracted_time.split(' ')[1]
    time = time.split('.')[0]
    DateTime = date + ' ' + time
    return DateTime

def round_values(mobile_df):
    mobile_df['x_mobile'] = [round(val,3) for val in mobile_df['x_mobile']]
    mobile_df['y_mobile'] = [round(val,3) for val in mobile_df['y_mobile']]
    mobile_df['z_mobile'] = [round(val,3) for val in mobile_df['z_mobile']]
    mobile_df['Abs_value'] = [round(val,3) for val in mobile_df['Abs_value']]
    return mobile_df
    
def mobile_processing(mobile_df, mobile_time_df):
    start = mobile_time_df['system time text'][0]
    start_DateTime = split_datetime(start)
    stop = mobile_time_df['system time text'][1]
    stop_DateTime = split_datetime(stop)
    date_format = "%Y-%m-%d %H:%M:%S"
    start_DateTime = datetime.strptime(start_DateTime, date_format)
    stop_DateTime = datetime.strptime(stop_DateTime, date_format)
    pd.options.mode.chained_assignment = None 
    mobile_df = mobile_df[::33]
    mobile_df['time'] = [round(time, 2) for time in mobile_df['Time (s)']]
    mobile_df['time'] = pd.to_timedelta(mobile_df['time'], unit='s')
    mobile_df['time'] = start_DateTime + mobile_df['time']
    mobile_df['time'] = mobile_df['time'].dt.round('s')
    del mobile_df['Time (s)']
    old_cols = mobile_df.columns
    new_cols = ['x_mobile', 'y_mobile', 'z_mobile', 'Abs_value', 'date']
    for i in range(len(new_cols)):
        mobile_df = mobile_df.rename(columns={old_cols[i]: new_cols[i]})
    mobile_df = round_values(mobile_df)
    return mobile_df
    
def save_only_3_values(df):
    date_counts = df['date'].value_counts()
    valid_dates = date_counts[date_counts == 3].index
    df = df[df['date'].isin(valid_dates)]
    return df
   
def add_msec(df):
    milliseconds = [300, 600, 900]
    df = df.copy()
    num_rows = len(df)
    repetitions = num_rows // len(milliseconds)
    pattern = np.tile(milliseconds, repetitions)
    remainder = num_rows % len(milliseconds)
    if remainder > 0:
        pattern = np.concatenate((pattern, milliseconds[:remainder]))
    df['date'] += pd.to_timedelta(pattern, unit='ms')
    return df

def merge_df(s_df, m_df):
    merge = pd.merge(m_df, s_df) 
    columns_titles = ["date","x_mobile","y_mobile", "z_mobile", "x", "y", "z", 'Abs_value']
    df=merge.reindex(columns=columns_titles)
    return df

def main():
    write_csv(log_filename='storage_data/sensors-2023-08-16.log', csv_filename='storage_data/sensors.csv')
    mobile_df = pd.read_csv('storage_data/Raw Data.csv')
    mobile_time_df = pd.read_csv('storage_data/time.csv')
    extract_df = pd.read_csv('storage_data/sensors.csv')
    sensor_df = sensor_processing(extract_df)
    sensor_df = conversion(sensor_df)
    mobile_df = mobile_processing(mobile_df, mobile_time_df)
    sensor_df = save_only_3_values(sensor_df)
    mobile_df = save_only_3_values(mobile_df)
    sensor_df = add_msec(sensor_df)
    mobile_df = add_msec(mobile_df)
    mobile_sensor_output = merge_df(sensor_df, mobile_df)

main()